<a href="https://colab.research.google.com/github/TheAECode/PRODIGY_ML_04/blob/main/PRODIGY_ML_04.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install tensorflow keras opencv-python scikit-learn matplotlib


In [2]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("gti-upm/leapgestrecog")

print("Path to dataset files:", path)

100%|██████████| 2.13G/2.13G [00:24<00:00, 92.3MB/s]

Extracting files...


Path to dataset files: /root/.cache/kagglehub/datasets/gti-upm/leapgestrecog/versions/1


In [9]:
import os
dataset_path = "/root/.cache/kagglehub/datasets/gti-upm/leapgestrecog/versions/1/leapGestRecog"

images = []
labels = []

for label_folder in sorted(os.listdir(dataset_path)):
    folder_path = os.path.join(dataset_path, label_folder)
    if os.path.isdir(folder_path):  # Ensure it’s a directory
        for image_file in os.listdir(folder_path):
            image_path = os.path.join(folder_path, image_file)
            if image_file.endswith(('.jpg', '.jpeg', '.png')):  # Validate image format
                try:
                    # Read and preprocess image
                    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
                    if image is None:
                        print(f"Skipping invalid file: {image_path}")
                        continue
                    image = cv2.resize(image, (64, 64))
                    images.append(image)
                    labels.append(label_folder)
                except Exception as e:
                    print(f"Error processing {image_path}: {e}")


In [10]:
print(f"Number of images loaded: {len(images)}")
print(f"Unique labels: {set(labels)}")


Number of images loaded: 0
Unique labels: set()


In [15]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator

def load_and_preprocess_data(data_path):
    datagen = ImageDataGenerator(
        rescale=1./255,
        validation_split=0.2,
        rotation_range=10,
        zoom_range=0.1
    )

    train_generator = datagen.flow_from_directory(
        data_path,
        target_size=(48, 48),
        batch_size=128,
        class_mode='categorical',
        subset='training'
    )

    validation_generator = datagen.flow_from_directory(
        data_path,
        target_size=(48, 48),
        batch_size=128,
        class_mode='categorical',
        subset='validation'
    )

    return train_generator, validation_generator

def create_model(num_classes):
    model = models.Sequential([
        layers.Input(shape=(48, 48, 3)),
        layers.Conv2D(32, 3, activation='relu', padding='same'),
        layers.MaxPooling2D(),
        layers.Conv2D(64, 3, activation='relu', padding='same'),
        layers.MaxPooling2D(),
        layers.Conv2D(64, 3, activation='relu'),
        layers.Flatten(),
        layers.Dense(64, activation='relu'),
        layers.Dropout(0.2),
        layers.Dense(num_classes, activation='softmax')
    ])

    model.compile(
        optimizer=tf.keras.optimizers.Adam(1e-3),
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )

    return model

def train_model(model, train_generator, validation_generator):
    return model.fit(
        train_generator,
        validation_data=validation_generator,
        epochs=3
    )

# Train
data_path = "/root/.cache/kagglehub/datasets/gti-upm/leapgestrecog/versions/1"
train_generator, validation_generator = load_and_preprocess_data(data_path)
model = create_model(len(train_generator.class_indices))
history = train_model(model, train_generator, validation_generator)

Found 32000 images belonging to 2 classes.
Found 8000 images belonging to 2 classes.
Epoch 1/3
250/250 ━━━━━━━━━━━━━━━━━━━━ 283s 1s/step - accuracy: 0.5028 - loss: 0.6939 - val_accuracy: 0.5000 - val_loss: 0.6932
Epoch 2/3
250/250 ━━━━━━━━━━━━━━━━━━━━ 295s 1s/step - accuracy: 0.4970 - loss: 0.6932 - val_accuracy: 0.5000 - val_loss: 0.6931
Epoch 3/3
250/250 ━━━━━━━━━━━━━━━━━━━━ 279s 1s/step - accuracy: 0.4961 - loss: 0.6932 - val_accuracy: 0.5000 - val_loss: 0.6932


In [17]:
import numpy as np
from tensorflow.keras.preprocessing import image
import cv2
import os

def test_single_image(model, image_path, target_size=(48, 48)):
    img = image.load_img(image_path, target_size=target_size)
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array /= 255.0

    prediction = model.predict(img_array)
    predicted_class = np.argmax(prediction)
    confidence = prediction[0][predicted_class]

    return predicted_class, confidence

# Create a dummy test image for the example
dummy_image = np.zeros((48, 48, 3), dtype=np.uint8)
cv2.imwrite("/content/Flower-bud-003.webp", dummy_image)


# Example usage
image_path = "/content/Flower-bud-003.webp"  # Use the dummy image path
predicted_class, confidence = test_single_image(model, image_path)
print(f"Predicted class: {predicted_class}")
print(f"Confidence: {confidence:.2f}")

# Clean up the dummy image
os.remove("/content/Flower-bud-003.webp")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 89ms/step
Predicted class: 1
Confidence: 0.50
